## VQE - Variational Quantum Eigensolver

- What is Optimization?

   Search for maxima or minima. (Machine Learning: from data to pattern recognition)
   
------------------

- What is Simulation?

  A sampling technique. (Physics and Engineering : from pattern(mathematical equation) to data generation))

### VQE problem statement:

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ Given a Hamiltonian H, solve the following optimization problem

$$
\min_{\theta}~~~ \langle \psi(\theta) |H| \psi (\theta) \rangle
$$

<img src="vqe.png" alt="Drawing" style="width: 700px; height 500px;"/>

<img src="https://miro.medium.com/max/3000/1*TvB2W5sqAJ4HXvIPHBKgqQ.png" alt="Drawing" style="width: 700px; height 500px;"/>

### Simple Example

In [9]:
from qiskit import *

* Lets us consider very simple $|\psi\rangle $ as probability vector (i.e. H is I matrix)

In [10]:
import numpy as np
np.random.seed(999999)
target_distr = np.random.rand(2)
# We now convert the random vector into a valid probability vector
target_distr /= sum(target_distr)

* Variational Form can be prepared with three paramentrs $\theta, \phi, \lambda $

$$
\begin{align}
    U3(\theta, \phi, \lambda) = \begin{pmatrix}\cos(\frac{\theta}{2}) & -e^{i\lambda}\sin(\frac{\theta}{2}) \\ e^{i\phi}\sin(\frac{\theta}{2}) & e^{i\lambda + i\phi}\cos(\frac{\theta}{2}) \end{pmatrix}
\end{align}
$$

In [11]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

* Classical Part: Cost function (Objective function)

In [15]:
from qiskit import Aer
from qiskit.compiler import transpile,assemble
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0])
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    t_qc = transpile(qc, backend)
    qobj = assemble(t_qc, shots=NUM_SHOTS)
    result = backend.run(qobj).result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

* Classical Part: Optimization

In [16]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# Obtain the output distribution using the final parameters
qc = get_var_form(ret[0])
t_qc = transpile(qc, backend)
qobj = assemble(t_qc, shots=NUM_SHOTS)
counts = backend.run(qobj).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])

Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.5248, 0.4752]
Output Error (Manhattan Distance): 0.030059881261160826
Parameters Found: [1.53560768 1.2072113  0.63106461]
